In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder


data = pd.read_csv('C:/Users/Jibin Im/Downloads/archive/fake reviews dataset.csv')

data.columns

Index(['category', 'rating', 'label', 'text_'], dtype='object')

In [2]:
data.head()

,category,rating,label,text_
0,Home_and_Kitchen_5,5.0,CG,"Love this! Well made, sturdy, and very comfor..."
1,Home_and_Kitchen_5,5.0,CG,"love it, a great upgrade from the original. I..."
2,Home_and_Kitchen_5,5.0,CG,This pillow saved my back. I love the look and...
3,Home_and_Kitchen_5,1.0,CG,"Missing information on how to use it, but it i..."
4,Home_and_Kitchen_5,5.0,CG,Very nice set. Good quality. We have had the s...


In [3]:
data['label'] = data['label'].map({'CG':1, 'OR':0})# 1 if computer generated, 0 if not
data['category'] = LabelEncoder().fit_transform(data['category'])
X = data.drop(columns='label')
y = data['label']

data.dropna()

,category,rating,label,text_
0,3,5.0,1,"Love this! Well made, sturdy, and very comfor..."
1,3,5.0,1,"love it, a great upgrade from the original. I..."
2,3,5.0,1,This pillow saved my back. I love the look and...
3,3,1.0,1,"Missing information on how to use it, but it i..."
4,3,5.0,1,Very nice set. Good quality. We have had the s...
...,...,...,...,...
40427,1,4.0,0,I had read some reviews saying that this bra r...
40428,1,5.0,1,I wasn't sure exactly what it would be. It is ...
40429,1,2.0,0,"You can wear the hood by itself, wear it with ..."
40430,1,1.0,1,I liked nothing about this dress. The only rea...


In [4]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

X = data['text_']
y = data['label']

vectorizer = TfidfVectorizer(stop_words='english')
X_tfidf = vectorizer.fit_transform(X)

X_tfidf_dense = X_tfidf.toarray()

X_train, X_test, y_train, y_test = train_test_split(X_tfidf_dense, y, test_size=0.33, random_state=42)

clf1 = LogisticRegression()#Accuracy: 0.8599 (approx due to random split)

clf2 = MultinomialNB()#using voting with logreg, Accuracy: 0.8665


XGBClassifier()#0.8350

GaussianNB()#Accuracy: 0.7428

clf3 = VotingClassifier(estimators=[('logreg', clf1), ('mnb', clf2)],
                        voting='soft',
                        weights=[0.5, 0.5])
clf3.fit(X_train, y_train)

pred = clf3.predict(X_test)
accuracy = accuracy_score(y_test, pred)
print(f'Accuracy: {accuracy:.4f}')

Accuracy: 0.8665


In [15]:
def createReview(vectorizer):
    text_ = input("Enter your review: ")

    X_tfidf = vectorizer.transform([text_])
    X_tfidf_dense = X_tfidf.toarray()

    return X_tfidf_dense
 
new_review = createReview(vectorizer)
prediction = clf3.predict(new_review)

print("Real Review" if prediction[0] == 0 else "Computer Generated Review")

Real Review
